# Inicialização do Navegador

In [5]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pprint


nav = webdriver.Chrome()

## Definição das funçoes de busca do Google Shopping e do Buscapé

In [13]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_resultados = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # entrar no google
    nav.get("https://www.google.com.br/")
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    # opção de busca por "class name"
    time.sleep(2)
    # entrar na aba shopping
    elementos = nav.find_elements("class name", "YmvwI")


    time.sleep(2)

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar as informações do produto
    resultado_pesquisa = nav.find_elements('class name', 'i0X6df')

    for resultado in resultado_pesquisa:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        
        # analizar se o produto tem algum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
                
        # analizar se o produto contem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        
        
        # filtro do valor
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+", "").replace("impostos", "")
            preco = float(preco)
                        
            if preco_minimo<= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_resultados.append((nome, preco, link))
    return lista_resultados

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_resultado = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar os resultados
    time.sleep(5)
    resultado_pesquisa = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    
    for resultado in resultado_pesquisa:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute("href")
        
        # analizar se o produto tem algum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
       
        # analizar se o produto contem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        
        # analisar a lista de ofertas 
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+", "").replace("impostos", "")
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_resultado.append((nome, preco, link))
                
    return lista_resultado
        
    
       

### Importar os dados do excel

In [14]:
# importar/ visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,2000,3500
1,rtx 3060,zota galax,3500,4500


### Construindo a nossa lista de ofertas

In [ ]:
# salvar as ofertas boas em um dataframe (tabela)
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:

    # pesquizar pelo produto
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    Lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if Lista_ofertas_google_shopping:
        tabel_google_shopping = pd.DataFrame(Lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabel_google_shopping])
    else:
        tabel_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

nav.quit
  
display(tabela_ofertas)



### Exportando para o Excel

In [16]:
# exportar pro excel
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

### Enviando para o E-mail

In [ ]:
# enviar por email o resultado da tabela / opção de envio Outlook

import win32com.client as win32

def send_email_with_attachment(to_address, subject, body):
    # Criar uma instância do Outlook
    outlook = win32.Dispatch('outlook.application')
    
    # Criar um novo e-mail
    mail = outlook.CreateItem(0)
    
    # Definir os parâmetros do e-mail
    mail.To = to_address
    mail.Subject = subject
    mail.Body = body
    
    # Enviar o e-mail
    mail.Send()

# Exemplo de uso
to_address = 'destinatario@exemplo.com'
subject = 'Assunto do Email'
body = f"""
<p>Prezados, </p>

<p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada </p>
{tabela_ofertas.to_html(index=False)}
<p>Att., </p>
<p>Marcio Luiz </p>
"""


if len(tabela_ofertas) > 0:
    send_email_with_attachment(to_address, subject, body)


# se necessario anexar algum arquivo, alterar codigo com attachment_path

